In [2]:
import pandas as pd
import numpy as np

In [ ]:
# IEA el_rate per region
# if region="SSA":
#    ($BaU_values.$E$6)
# elif region="DA":
#        ($BaU_values.$J$6)
# elif region="LA":
#    ($BaU_values.$G$6)
# else:
#    0

In [95]:
shs_sales_volumes = pd.read_csv('daten/shs_sales_volumes.csv', comment='#')
# compute the average of the product categories 5 to 7
shs_sales_volumes['tot_5-7'] = shs_sales_volumes[['5', '6', '7']].sum(axis=1)

shs_power_categories = pd.read_csv('daten/shs_power_per_product_categories.csv', comment='#')
# compute the average power for each category
shs_power_categories['power_av'] = shs_power_categories[['power_low', 'power_high']].mean(axis=1)

# select only the average power for the product categories 5 to 7
cat_power_av_5_7 = shs_power_categories['power_av'].loc[shs_power_categories.category >= 5].values
# compute the average power per region (multiply the number units sold per product categories by the average power of the respective category)
power_av_5_7 = shs_sales_volumes[['5', '6', '7']].values * cat_power_av_5_7
# compute the average unit power per region (divide the sum over the categories by the total unit sold)
shs_sales_volumes['weighted_tot_5-7 [W]'] = power_av_5_7.sum(axis=1) / shs_sales_volumes['tot_5-7'].values

/home/Pierre.Duc/.virtualenvs/ndc-map/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  


In [106]:
# Global parameters
# variable
tier
invest_shs
invest_mg
# fixed
weight_rise
weight_experts

67.2942122656684

In [115]:
ELECTRIFICATION_OPTIONS =['grid', 'mg', 'shs']
SENARII = ['bau', 'se4all', 'prog']

In [111]:
#Fixed inputs (from a csv)
# country,country_iso,region,hh_av_size,pop_2017,pop_2030,dark_rate,hh_grid_share,hh_yearly_electricity_consumption,gdp_per_capita,mobile_money_2014,mobile_money_2017,ease_doing_business_index
country
country_iso
region
dark_rate
electrification_rate
pop_shs_share
pop_mg_share
pop_grid_share
pop_2017
pop_2030

hh_grid_share
hh_mg_share
shs_unit_av_capacity
hh_yearly_electricity_consumption
iea_regional_electricity_coverage

ELECTRIFICATION_OPTIONS =['grid', 'mg', 'shs']
SENARII = ['bau', 'se4all', 'prog']

MIN_TIER_LEVEL = 3
MIN_RATED_CAPACITY = {3: 200, 4: 800, 5: 2000} # index is TIER level [W]
MIN_ANNUAL_CONSUMPTION = {3: 365, 4: 1250, 5: 3000} # index is TIER level [kWh/a]
# Find names for function on TIER lines 6 and 7

pop_rel_growth = pop_2030 / pop_2017
pop_dark_2017 = pop_2017 * dark_rate

hh_grid_tier_yearly_electricity_consumption  # Grid_Tier_adjusted_HH_el_consumption [kWh/a]
#=S3*WENN(AA3<$Tiers.$B$2;$Tiers.$B$2;$'Data_+_GIS'.AA3)
if hh_yearly_electricity_consumption < MIN_ANNUAL_CONSUMPTION[MIN_TIER_LEVEL]:
    MIN_ANNUAL_CONSUMPTION[MIN_TIER_LEVEL] * hh_grid_share
else:
    hh_yearly_electricity_consumption * hh_grid_share

hh_mg_tier_yearly_electricity_consumption  # MG_Tier_adjusted_HH_el_consumption [kWh/a]
#=WENN(AA3<$Tiers.$B$2/T3;$Tiers.$B$2;$'Data_+_GIS'.AA3*T3)
if hh_yearly_electricity_consumption < MIN_ANNUAL_CONSUMPTION[MIN_TIER_LEVEL] / hh_mg_share:
    MIN_ANNUAL_CONSUMPTION[MIN_TIER_LEVEL]
else:
    hh_yearly_electricity_consumption * hh_mg_share

    
hh_grid_tier_peak_demand  # Grid peak demand grid [kW/HH], interpolated per Tier
#=WENN(UND(AB3>=$Tiers.$C$5;AB3<$Tiers.$D$5);AB3*$Tiers.$D$6+$Tiers.$D$7;WENN(UND(AB3>=$Tiers.$B$5;AB3<$Tiers.$C$5);AB3*$Tiers.$C$6+$Tiers.$C$7))/1000
if hh_grid_tier_yearly_electricity_consumption >= MIN_ANNUAL_CONSUMPTION[4] and hh_grid_tier_yearly_electricity_consumption < MIN_ANNUAL_CONSUMPTION[5]:
    hh_grid_tier_yearly_electricity_consumption*$Tiers.$D$6+$Tiers.$D$7/1000
    #$Tiers.$D$6 = Ratio of MIN_RATED_CAPACITY and MIN_ANNUAL_CONSUMPTION between TIER 5 and TIER 4
    #$Tiers.$D$7 = MIN_RATED_CAPACITY[4] - MIN_ANNUAL_CONSUMPTION[4] * $Tiers.$D$6
else:
    if hh_grid_tier_yearly_electricity_consumption>=MIN_ANNUAL_CONSUMPTION[3] and hh_grid_tier_yearly_electricity_consumption<MIN_ANNUAL_CONSUMPTION[4]:
    #$Tiers.$C$6 = Ratio of MIN_RATED_CAPACITY and MIN_ANNUAL_CONSUMPTION between TIER 4 and TIER 3
    #$Tiers.$C$7 = MIN_RATED_CAPACITY[3] - MIN_ANNUAL_CONSUMPTION[3] * $Tiers.$C$6
        hh_grid_tier_yearly_electricity_consumption*$Tiers.$C$6+$Tiers.$C$7/1000

hh_mg_tier_peak_demand  # MG peak demand grid [kW/HH], interpolated per Tier
#=WENN(UND(AC3>=$Tiers.$C$5;AC3<$Tiers.$D$5);AC3*$Tiers.$D$6+$Tiers.$D$7;WENN(UND(AC3>=$Tiers.$B$5;AC3<$Tiers.$C$5);AC3*$Tiers.$C$6+$Tiers.$C$7))/1000
if hh_mg_tier_yearly_electricity_consumption>=MIN_ANNUAL_CONSUMPTION[4] and hh_mg_tier_yearly_electricity_consumption < MIN_ANNUAL_CONSUMPTION[5]:
    hh_mg_tier_yearly_electricity_consumption*$Tiers.$D$6+$Tiers.$D$7/1000
    #$Tiers.$D$6 = Ratio of MIN_RATED_CAPACITY and MIN_ANNUAL_CONSUMPTION between TIER 5 and TIER 4
    #$Tiers.$D$7 = MIN_RATED_CAPACITY[4] - MIN_ANNUAL_CONSUMPTION[4] * $Tiers.$D$6
else:
    if hh_mg_tier_yearly_electricity_consumption>=MIN_ANNUAL_CONSUMPTION[3] and hh_mg_tier_yearly_electricity_consumption<MIN_ANNUAL_CONSUMPTION[4]:
    #$Tiers.$C$6 = Ratio of MIN_RATED_CAPACITY and MIN_ANNUAL_CONSUMPTION between TIER 4 and TIER 3
    #$Tiers.$C$7 = MIN_RATED_CAPACITY[3] - MIN_ANNUAL_CONSUMPTION[3] * $Tiers.$C$6
        hh_mg_tier_yearly_electricity_consumption*$Tiers.$C$6+$Tiers.$C$7/1000


shs_unit_av_capacity  # SHS average capacity [Wp per sold unit]
#=WENN(C3="SSA";$'SHS sales volume'.$V$12;WENN(C3="DA";$'SHS sales volume'.$V$17;WENN(C3="LA";$'SHS sales volume'.$V$20;0)))
if region=="SSA":
    shs_sales_volumes.loc[shs_sales_volumes.region == 'SSA']['weighted_tot_5-7 [W]'].values[0]
elif region=="DA":
    shs_sales_volumes.loc[shs_sales_volumes.region == 'South Asia']['weighted_tot_5-7 [W]'].values[0]
elif region=="LA":
    shs_sales_volumes.loc[shs_sales_volumes.region == 'Latin America']['weighted_tot_5-7 [W]'].values[0]
else:
    0






SyntaxError: invalid syntax (<ipython-input-111-541ee21daff5>, line 45)

In [148]:
bau_data = pd.read_csv('daten/bau.csv', comment='#')
bau_data = bau_data.set_index('region')

In [149]:
bau_data

,region_description,el_rate_2030_IEA_NewPol,shs_share,mg_share,grid_share
region,,,,,
SSA,Sub Saharan Africa,0.64,0.340708,0.163717,0.495575
DA,Developing Asia,0.99,0.223602,0.329193,0.447205
LA,Latin America,0.99,0.142857,0.285714,0.571429


In [154]:
#bau_pop_newly_electrified = iea_regional_electricity_coverage * pop_newly_electrified_2030

#pop_newly_electrified_2030 = pop_rel_growth * pop_dark_2017

region = 'SSA'

bau_pop_grid_share = bau_data.loc[region]['grid_share']  # BaU Grid share [%population] 
#=WENN($C4="SSA";$BaU_values.$E$5; WENN($C4="DA";$BaU_values.$F$5;WENN($C4="LA";$BaU_values.$G$5;0)))
# if region="SSA":
#    ($BaU_values.$E$5)
# elif region="DA":
#        ($BaU_values.$F$5)
# elif region="LA":
#    ($BaU_values.$G$5)
# else:
#    0

bau_pop_mg_share = bau_data.loc[region]['mg_share']  # BaU MG share [%population]
#=WENN($C4="SSA";$BaU_values.$E$4; WENN($C4="DA";$BaU_values.$F$4;WENN($C4="LA";$BaU_values.$G$4;0)))
# if region="SSA":
#    ($BaU_values.$E$4)
# elif region="DA":
#        ($BaU_values.$F$4)
# elif region="LA":
#    ($BaU_values.$G$4)
# else:
#    0

bau_pop_shs_share = bau_data.loc[region]['shs_share']  # BaU SHS share [%population]
#=WENN($C4="SSA";$BaU_values.$E$3; WENN($C4="DA";$BaU_values.$F$3;WENN($C4="LA";$BaU_values.$G$3;0)))
# if region="SSA":
#    ($BaU_values.$E$3)
# elif region="DA":
#        ($BaU_values.$F$3)
# elif region="LA":
#    ($BaU_values.$G$3)
# else:
#    0

0.340707965

In [159]:
bau_data.iloc[0:3]
regions=['SSA', 'DA', 'LA']
bau_data.loc[regions]
bau_data.loc[regions]['shs_share'].to_list()

[0.340707965, 0.223602484, 0.14285714300000002]

In [124]:
df = pd.DataFrame({'A': [0, 1, 2, 3, 4],
                    'B': [5, 6, 7, 8, 9],
                    'C': ['a', 'a', 'b', 'c', 'c']})

df['D'] = df['C'].replace(['a','b', 'c'], [1.2, 0, -3])#.replace('b',0).replace
df

,A,B,C,D
0,0,5,a,1.2
1,1,6,a,1.2
2,2,7,b,0.0
3,3,8,c,-3.0
4,4,9,c,-3.0


In [ ]:

def extract_results_senario(df, senario, regions=None):
    
    if region is None:
        regions=['SSA', 'DA', 'LA']
    
    for opt in ELECTRIFICATION_OPTIONS:
        # not valid for other senario than bau at the moment
        # create a columns with regional electrification option shares 
        df['temp_%s' % opt] = df['region'].replace(regions, bau_data.loc[regions]['%s_share' % opt].to_list())
        
        # predicted number of people getting access to electricity (regional detail level)
        df['pop_get_%s_2030' % opt] = df['%s__pop_newly_electrified'] * df['temp_%s' % opt]
        
        # predicted number of household getting access to electricity (regional detail level)
        df['hh_get_%s_2030' % opt] = df['pop_get_%s_2030' % opt] / hh_av_size
        
        # predicted power (in kW) that the access to electricity will represent (regional detail level)
        # the analysis is based on the peak demand for the grid and mg senarii, and the average
        # power of solar panel for shs senario
        if opt in ('grid' , 'mg'):
            df['hh_%s_capacity' % opt] = df['hh_get_%s_2030' % opt] * df['hh_%s_tier_peak_demand' % opt]
        else:
            df['hh_%s_capacity' % opt] = df['hh_get_%s_2030' % opt] * df['shs_unit_av_capacity'] / 1000
            
    
    #pop_get_grid_2030 = BaU Grid share [%population] * bau_pop_newly_electrified
    #pop_get_mg_2030 = BaU MG share [%population] * bau_pop_newly_electrified
    #pop_get_shs_2030 = BaU SHS share [%population] * bau_pop_newly_electrified

    # the average num people per household is NOT the 2030 one...
    #hh_get_grid_2030 = pop_get_grid_2030 / hh_av_size
    #hh_get_mg_2030 = pop_get_mg_2030 / hh_av_size
    #hh_get_shs_2030 = pop_get_shs_2030 / hh_av_size

    # the average num people per household is NOT the 2030 one...
    # results in kW
    #hh_grid_capacity = hh_get_grid_2030 *  Grid peak demand grid [kW/HH], interpolated per Tier 
    #hh_mg_capacity = hh_get_mg_2030 *  MG peak demand grid [kW/HH], interpolated per Tier  
    #hh_shs_capacity = hh_get_shs_2030 * (SHS average capacity [Wp per sold unit] / 1000)




In [190]:
df = pd.read_csv('daten/raw_data.csv')

In [198]:
df.loc[:,['mobile_money_2014', 'ease_doing_business_index', 'hh_av_size']].sum(axis=1)

0     187.54
1     156.81
2     163.02
3     148.73
4     178.53
5      75.91
6     190.02
7     147.64
8     177.22
9     189.39
10    180.32
11    195.44
12    150.60
13    117.63
14    172.21
15    159.32
16    187.95
17     85.82
18    113.93
19    186.44
20    113.00
21    134.82
22    112.48
23    131.73
24    184.00
25    117.30
26    168.74
27    148.82
28    171.20
29    137.40
30    174.16
31    145.54
32    105.50
33    167.14
34    173.62
35    103.40
36     53.80
37    107.74
38     66.48
39    164.72
40    161.36
41    152.64
42      6.27
43    193.02
44    188.86
45    154.61
46    144.22
47    127.05
48    176.70
49     76.34
50    102.22
51    159.32
dtype: float64

In [ ]:

RISE_INDICES = ['rise_grid', 'rise_mg', 'rise_shs']
SHIFT_MENTI = ['shift_menti_mg', 'shift_menti_shs']
#=WENN(SUMME(S22:W22)=0;0;S22*$RT_shift_factors.$O$2/(SUMME($S22:$U22)*$RT_shift_factors.$O$2+SUMME($V22:$W22)*$RT_shift_factors.$P$2))

SUMME(S22:W22) --> df.loc[:,RISE_INDICES + SHIFT_MENTI].sum(axis=1)
SUMME($S22:$U22) --> df.loc[:,RISE_INDICES].sum(axis=1)
SUMME($V22:$W22) --> df.loc[:,SHIFT_MENTI].sum(axis=1)
answer = 1
if SUMME(S22:W22)=0;
    answer = 0
else:S22*$RT_shift_factors.$O$2/(SUMME($S22:$U22)*$RT_shift_factors.$O$2+SUMME($V22:$W22)*$RT_shift_factors.$P$2))
df['shift_grid_share'] = df['']

df.loc[:,['mobile_money_2014', 'ease_doing_business_index', 'hh_av_size']].sum(axis=1)

#=WENN(SUMME(S22:W22)=0;0;(T22*$RT_shift_factors.$O$2+V22*$RT_shift_factors.$P$2)/(SUMME($S22:$U22)*$RT_shift_factors.$O$2+SUMME($V22:$W22)*$RT_shift_factors.$P$2))

df['shift_grid_to_mg_share'] = df['']

#=WENN(SUMME(S23:W23)=0;0;(U23*$RT_shift_factors.$O$2+W23*$RT_shift_factors.$P$2)/(SUMME($S23:$U23)*$RT_shift_factors.$O$2+SUMME($V23:$W23)*$RT_shift_factors.$P$2))

df['shift_grid_to_shs_share'] = df['']

#=WENN(Y24>=X24;$Y24*D24;0)

df['shift_pop_grid_to_mg'] = df['']

#=WENN(Z25>=X25;Z25*$D25;0)

df['shift_pop_grid_to_shs'] = df['']

In [ ]:
#=WENN(Q22<4,5;0;WENN(Q22<=9;0,5;WENN(Q22>9;1;"NA")))

def map_weak_grid_class(weak_grid_idx):
    """Assign an index value to differentiate weak grid"""
    answer = 1
    if weak_grid_idx <= 4.5:
        answer = 0.5
    if weak_grid_idx <= 9:
        answer = 0
    return answer

df['weak_grid_class'] = df['weak_grid_index'].map(map_weak_grid_class, na_action='ignore')

In [195]:
#=WENN(O25<26;0;WENN(O25<=33;0,5;WENN(O25>33;1;"NA")))

def map_corruption_class(corruption_idx):
    """Assign an index value to differentiate corruption"""
    answer = 1
    if corruption_idx <= 33:
        answer = 0.5
    if corruption_idx <= 26:
        answer = 0
    return answer

df['corruption_class'] = df['corruption_index'].map(map_corruption_class, na_action='ignore')



In [193]:
#=WENN(M22<131;0;WENN(M22<=164;0,5;WENN(M22>164;1;"NA")))

def map_ease_doing_business_class(business_ease):
    """Assign an index value to differentiate ease of doing business"""
    answer = 1
    if business_ease <= 164:
        answer = 0.5
    if business_ease <= 131:
        answer = 0
    return answer

df['ease_doing_business_class'] = df['ease_doing_business_index'].map(map_ease_doing_business_class, na_action='ignore')


In [174]:
#Se4All

def map_gdp_class(gdp_per_capita):
    """Assign an index value to differentiate gdp per capita"""
    answer = 1
    if gdp_per_capita < 1500:
        answer = 0.5
    if gdp_per_capita < 700:
        answer = 0
    return answer
    
df['gdp_class'] = df['gdp_per_capita'].map(map_gdp_class, na_action='ignore')

In [188]:
#=WENN(K5<=0,12;0;WENN(K5<=0,21;0,5;1))

def map_mobile_money_class(mobile_money):
    """Assign an index value to differentiate mobile_money"""
    answer = 1
    if mobile_money <= 0.21:
        answer = 0.5
    if mobile_money <= 0.12:
        answer = 0
    return answer

df['mobile_money'] = df['mobile_money_2017'].fillna(df['mobile_money_2014'])
df['mobile_money_class'] = df['mobile_money'].map(map_mobile_money_class, na_action='ignore').fillna(0)

In [ ]:
map('I am a {}'.format, na_action='ignore')
#=WENN(G4<700;0;WENN(G4<1500;0,5;1))
#if G4<700;
    0;
elif G4<1500;
0,5;
else
1)